In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [ ]:
import os
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    def extract_last_bracketed(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None
    df[write_col] = df[read_col].apply(extract_last_bracketed)
    return df

def validate_text_column(df, df_reference):
    reference_texts = set(df_reference['text'].unique())
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, 'output', 'extracted_label')
    return df

for i in [5]:
    print('\n\nMUMER', i, '\n\n')
    
    model_tag = 'llama3'
    dataset_tag = '20_newsgroups'
    prompt = 'cot_random_samples_cohere'
    temp = 0.3
    number_of_experimet = i
    selected_samples = f'{model_tag}_random42_80'
    
    model = get_model_by_tag(model_tag, token, temp)
    prompt_txt = get_prompt_template(dataset_tag, prompt)
    examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

    output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    try:
        df_old = pd.read_csv(output_path)
    except FileNotFoundError:
        df_old = pd.DataFrame(columns=['text', 'label', 'output'])
        
    df_old = validate_text_column(df_old, examples_for_prompt)

    # Dodaj 'original_index', żeby móc później przywrócić kolejność
    full_dataset = get_dataset(dataset_tag=dataset_tag)
    full_dataset = full_dataset.reset_index().rename(columns={'index': 'original_index'})

    # Wczytaj stare wyniki lub stwórz pusty DataFrame
    try:
        df_old = pd.read_csv(output_path)
    except FileNotFoundError:
        df_old = pd.DataFrame(columns=['text', 'label', 'output', 'error'])
        
    # Do zaannotowania:
    # - brak tekstu w df_old
    # - lub w df_old jest 'error' niepuste dla danego textu
    if 'error' not in df_old.columns:
        df_old['error'] = None
    texts_with_errors = df_old[df_old['error'].notna() & (df_old['error'].astype(str) != '')]['text']
    texts_already_done = set(df_old['text']) - set(texts_with_errors)
    df_to_annotate = full_dataset[~full_dataset['text'].isin(texts_already_done)]

    print('Do zaannotowania:', len(df_to_annotate))


    if not df_to_annotate.empty:
        annotator = LLMAnnotator(
            model=model,
            dataset=df_to_annotate,
            examples_for_prompt=examples_for_prompt,
            prompt_template=prompt_txt,
            column_text="text",
            column_label="label",
            column_output="output"
        )
        df_new = annotator.get_results()
        df_new = validate_text_column(pd.DataFrame(df_new), examples_for_prompt)

        # Łączenie z zachowaniem indeksu
        
        df_new = df_new.merge(df_to_annotate[['text', 'original_index']], on='text', how='left')

        df_old = df_old.merge(full_dataset[['text', 'original_index']], on='text', how='left')
        
        df_combined = pd.concat([df_old, df_new], ignore_index=True)
        df_combined.drop_duplicates(subset='original_index', keep='last', inplace=True)
        df_combined.sort_values(by='original_index', inplace=True)
        
        # Usuń kolumnę 'error', jeśli wszystkie wartości są puste lub puste stringi
        if 'error' in df_combined.columns and df_combined['error'].replace('', pd.NA).isna().all():
            df_combined.drop(columns='error', inplace=True)

        df_combined.to_csv(output_path, index=False)
        print("Zaktualizowano plik:", output_path)

    else:
        print("Brak nowych przykładów do anotacji.")




MUMER 5 


Do zaannotowania: 749


/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: Let's break down the text step by step:

1. **Summarize the main topic of the text**: The main topic of the text appears to be a discussion about a "dragless satellite" and its orbital parameters, specifically in the context of laser geodesy missions.

2. **Identify any key terms or phrases that hint at a specific category**: Key terms and phrases include "satellite", "laser geodesy", "orbital parameters", and "dragless". These terms are related to space and astronomy.

3. **Compare the topic to the definitions of each category**: Based on the key terms and phrases, the topic seems to fit best with categories related to space and astronomy.

4. **Choose the most appropriate category based on your reasoning**: Given the focus on satellites, orbital parameters, and laser geodesy, the most appropriate category is [sci.space].

So, the final answer is: [sci.space]
Nr: 1 Predicted label: Let's analyze the text step by step:

**Step 1: Summarize the main topic of the t

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. The content length of 412840299 bytes exceeds the maximum size limit of 20971520 bytes. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by SSLError(SSLError(5, '[SYS] unknown error (_ssl.c:2417)')))"))
Content-Length: 412840299
API Key: lsv2_********************************************7etrace=c58187d9-0f42-4f0a-bb0d-40df0532f764,id=c58187d9-0f42-4f0a-bb0d-40df0532f764


In [5]:
output_path

'./results/cot_random_samples_cohere_temp0.3/llama3/20_newsgroups_llama3_cot_random_samples_cohere_llama3_random42_80_temp0.3_exp5.csv'

In [6]:
df_combined

,text,output,logprobs,top_logprobs,original_label,original_index,was_in_selected_samples,extracted_label
0,article ashish arora writes excerpts netnewssc...,Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",talk.politics.misc,0,NaN,NaN
1,gateway telepath modem month actually one woul...,Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",comp.sys.ibm.pc.hardware,1,NaN,NaN
2,anybody provide advice concerning following tw...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",sci.med,2,NaN,NaN
3,article mike silverman writes anybody know goi...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",rec.sport.baseball,3,NaN,NaN
4,article stich christian e writes installed mot...,Let's break down the text step by step:\n\n1. ...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",sci.electronics,4,NaN,NaN
...,...,...,...,...,...,...,...,...
2019,david sternlight writes article karl barrus wr...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",sci.crypt,1995,False,sci.crypt
2020,hello im looking information alphanumeric page...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",sci.electronics,1996,False,sci.electronics
2021,john r daker writes would like offocially nomi...,Let's analyze the text step by step.\n\n1. Sum...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",rec.motorcycles,1997,False,rec.motorcycles
2022,article writes looking information concerning ...,Let's break down the text step by step:\n\n1. ...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",sci.space,1998,False,sci.space
